In [3]:
import requests
import zipfile
import shutil
import os
import pandas as pd
import sqlite3
import urllib3
from urllib3.exceptions import InsecureRequestWarning

# Desabilitar o aviso específico
urllib3.disable_warnings(InsecureRequestWarning)

# URLs dos arquivos ZIP
urls = [
    'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_brasil_2019.zip',
    'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_uf_2019.zip',
    'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_municipios_2019.zip'
]

# Diretório para salvar e extrair os arquivos
download_dir = './tmp/'
data_dir = './data/'
extract_dir = './data/raw/'
database_dir = './data/database.db'

if os.path.exists(download_dir):
    shutil.rmtree(download_dir)
    
os.makedirs(download_dir, exist_ok=True)

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
    
os.makedirs(data_dir, exist_ok=True)

if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)
    
os.makedirs(extract_dir, exist_ok=True)
    
    
# Extensão do arquivo que você deseja extrair
file_extension = '.xlsx'

for url in urls:
    # Nome do arquivo ZIP a ser salvo localmente
    local_zip_file = os.path.join(download_dir, url.split('/')[-1])
    
    # Fazendo o download do arquivo ZIP sem verificar o certificado SSL
    response = requests.get(url, verify=False)
    with open(local_zip_file, 'wb') as file:
        file.write(response.content)
    
    # Verificando se o arquivo foi baixado corretamente
    if os.path.getsize(local_zip_file) > 0:
        try:
            # Tentando abrir o arquivo ZIP para verificar sua integridade
            with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
                found = False
                # Percorrendo os arquivos no ZIP
                for file_name in zip_ref.namelist():
                    # Verificando se o arquivo termina com a extensão desejada
                    if file_name.endswith(file_extension):
                        # Extraindo o arquivo
                        zip_ref.extract(file_name, extract_dir)
                        print(f'Arquivo {file_name} extraído para {extract_dir}')
                        found = True
                
                if not found:
                    print(f'Erro: Nenhum arquivo com extensão {file_extension} encontrado no arquivo ZIP {local_zip_file}')
        
        except zipfile.BadZipFile:
            print(f"Erro: {local_zip_file} não é um arquivo ZIP válido")
    
    else:
        print(f"Erro: {local_zip_file} está vazio ou não foi baixado corretamente")

# Verificando os arquivos extraídos
extracted_files = os.listdir(extract_dir)

# Conectar ao banco de dados SQLite (isso criará o arquivo exemplo.db se não existir)
conn = sqlite3.connect(database_dir)


columns = [
    [
    'Ano', 'Região', 'UF', 'Código do município', 'Nome do município', 'Dependência administrativa', 'Escolaridade do docente', 'Número de docentes Censo Escolar1',
    '% de docentes localizados na RAIS', '1º quartil', 'Mediana', 'Média', '3º quartil',
    'Desvio padrão', 'Carga horária média semanal', 'Remuneração média padronizada para 40h semanais em R$'
    ], # Municipios collumns
    
    [
    'Ano', 'Região', 'UF', 'Dependência administrativa', 'Escolaridade do docente', 'Número de docentes Censo Escolar1',
    '% de docentes localizados na RAIS', '1º quartil', 'Mediana', 'Média', '3º quartil',
    'Desvio padrão', 'Carga horária média semanal', 'Remuneração média padronizada para 40h semanais em R$'
    ], # UFs collumns
    [
    'Ano', 'Abrangência', 'Dependência administrativa', 'Escolaridade do docente', 'Número de docentes Censo Escolar1',
    '% de docentes localizados na RAIS', '1º quartil', 'Mediana', 'Média', '3º quartil',
    'Desvio padrão', 'Carga horária média semanal', 'Remuneração média padronizada para 40h semanais em R$'
    ] # Brasil collumns
    
]

drop_collumns = [
    ['Ano'],
    ['Ano'],
    ['Ano', 'Abrangência']
]


table_names = [
    'wage_per_city',
    'wage_per_state',
    'wage_brasil'
]

for i in range(0, 3) :
    dataframe = pd.read_excel(f'{extract_dir}/{extracted_files[i]}/{extracted_files[i]}{file_extension}',
    skiprows=9, header=None, skipfooter=3)
    
    dataframe.columns = columns[i]
    
    dataframe_filtered = dataframe.drop(drop_collumns[i], axis=1)
    
    dataframe_filtered.to_sql(table_names[i], conn, index=False, if_exists='replace')



Arquivo Remuneracao_docentes_Brasil_2019/Remuneracao_docentes_Brasil_2019.xlsx extraído para ./data/raw/
Arquivo Remuneracao_docentes_UF_2019/Remuneracao_docentes_UF_2019.xlsx extraído para ./data/raw/
Arquivo Remuneracao_docentes_Municipios_2019/Remuneracao_docentes_Municipios_2019.xlsx extraído para ./data/raw/


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': Error binding parameter 0 - probably unsupported type.